Putting it all together!

In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import random
import os
from time import sleep
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def get_info(title, author):
    '''Given a title and author of a book in the list,
    returns a dictionary of prosecraft's analysis about the book.'''
    
    #Get rid of special characters in URL
    chars_to_remove = [':', '’', '.', ",", '“', '”']
    info = {'title': title, 'author': author}
    URL = f"{author}/{title}/"
    for char in chars_to_remove: 
        URL = URL.replace(char, '')
    URL = URL.replace('&', 'and').replace(' ','-').lower()
    URL = "http://prosecraft.io/library/" + URL
    
    #Get data from Prosecraft and turn it into a dict
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    headings = soup.find_all("div", {"class": "book-info-metric-heading"})
    values = soup.find_all("div", {"class": "book-info-metric-value"})
    for heading, value in zip(headings, values):
        info[heading.text] = float(value.text.strip('%').replace(',',''))
    return info

In [52]:
def get_genres(title, author):
    book_id = -1
    
    #Put the title into the search bar
    browser.get('http://www.goodreads.com/search?q=&qid=')
    search_book = browser.find_element(By.ID, value='search_query_main')
    search_book.send_keys(title)
    search_book.submit()
    sleep(20)
    itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
    img = browser.find_elements(By.CLASS_NAME, value="bookCover")
    book_list = list()
    
    #Search the first page of results for the author's name
    #Use only first author if multiple
    for i in range(len(itemqueue)):
        book_list.append(itemqueue[i].text.split('\n'))
        book_list_ap = list()
    for i in range(0, len(book_list)):
         book_list_ap.append((book_list[i][0],book_list[i][1],img[i].get_property("src")))
    for book in book_list_ap:
        if f"by {author.split(' &')[0]}" in book[1]:
            book_id = book[2].split('/')[-1].split('.')[0]
            break
            
    #Sometimes, a book's title is so common that the correct version isn't on the first page
    #If that happens, try putting the author into the search bar
    #And searching the first page for the correct title
    
    if book_id == -1:
        browser.get('http://www.goodreads.com/search?q=&qid=')
        search_book = browser.find_element(By.ID, value='search_query_main')
        search_book.send_keys(author)
        search_book.submit()
        sleep(20)
        itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
        img = browser.find_elements(By.CLASS_NAME, value="bookCover")
        book_list = list()
        for i in range(len(itemqueue)):
            book_list.append(itemqueue[i].text.split('\n'))
            book_list_ap = list()
        for i in range(0, len(book_list)):
             book_list_ap.append((book_list[i][0],book_list[i][1],img[i].get_property("src")))
        for book in book_list_ap:
            if title in book[0]:
                book_id = book[2].split('/')[-1].split('.')[0]
                break
    
    if book_id == -1:
        return []
        
        
    book_url = f'https://www.goodreads.com/book/show/{book_id}'
    browser.get(book_url)
    genres = browser.find_elements(By.XPATH, value="//span[contains(@class, 'BookPageMetadataSection__genreButton')]")
    sleep(20)
    return [genre.text for genre in genres]

In [4]:
with open('book_list.json', 'r') as lst:
    book_list = json.load(lst)


In [5]:
indices = random.sample(range(0, len(book_list)), 10)
books = []
for index in indices:
    books.append(book_list[index])

In [6]:
sample = []

for book in books:
    sample.append(get_info(book['t'], book['a']))
    

In [7]:
df = pd.DataFrame(sample)

In [8]:
df

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs
0,In the Shadow of the Ivory Tower,Davarian L. Baldwin,71566.0,32.32,4.21,2.34,1.05,1.30
1,The Girl in the Woods,Patricia MacDonald,84548.0,39.70,8.65,3.06,1.19,1.87
2,The Money Shot,Stuart Woods & Parnell Hall,57614.0,35.07,10.15,2.48,0.83,1.65
3,Outermost,Blaze Ward,41778.0,43.56,7.65,3.69,1.47,2.23
4,A Journey,Tony Blair,299371.0,18.43,8.93,4.04,1.75,2.29
5,Hero of the Empire,Candice Millard,104327.0,41.54,7.46,3.32,1.52,1.80
6,You Won’t Know I’m Gone,Kristen Orlando,82440.0,55.72,6.90,2.77,1.09,1.68
7,Cut,George Pelecanos,68845.0,51.82,7.17,2.13,0.68,1.45
8,Night Prey,John Sandford,96626.0,55.93,7.53,2.21,0.73,1.48
9,Interviewing the Dead,David Field,55861.0,35.57,7.89,3.72,1.61,2.11


In [9]:
#Fill in my Goodreads username and password from the .env
user_name = os.environ.get('USER')
password = os.environ.get('PASSWORD')

#This is just the URL I get when I go to goodreads and select log in by email.
login_url = os.environ.get('URL')

In [10]:
#Here I start up a headless Firefox browser through Selenium
s = Service("geckodriver.exe")
opts=Options()
opts.add_argument('-headless')
browser = webdriver.Firefox(service=s)
browser.get(login_url)

In [11]:
# Here I log into goodreads
log_email = browser.find_element(By.ID, value="ap_email")
log_pwd = browser.find_element(By.ID, value="ap_password")
log_email.send_keys(user_name)
log_pwd.send_keys(password)
log_pwd.submit()
sleep(5)

In [46]:
df['genres'] = df.apply(lambda x: get_genres(x['title'], x['author']), axis=1)

In [47]:
df

,title,author,total words,vividness,passive voice,all adverbs,ly-adverbs,non-ly-adverbs,genres
0,In the Shadow of the Ivory Tower,Davarian L. Baldwin,71566.0,32.32,4.21,2.34,1.05,1.30,"[Nonfiction, Education, Politics, Urban Planni..."
1,The Girl in the Woods,Patricia MacDonald,84548.0,39.70,8.65,3.06,1.19,1.87,"[Mystery, Thriller, Suspense, Fiction, Crime, ..."
2,The Money Shot,Stuart Woods & Parnell Hall,57614.0,35.07,10.15,2.48,0.83,1.65,"[Mystery, Fiction, Thriller, Crime, Audiobook,..."
3,Outermost,Blaze Ward,41778.0,43.56,7.65,3.69,1.47,2.23,[Science Fiction]
4,A Journey,Tony Blair,299371.0,18.43,8.93,4.04,1.75,2.29,"[Politics, Biography, Nonfiction, Autobiograph..."
5,Hero of the Empire,Candice Millard,104327.0,41.54,7.46,3.32,1.52,1.80,"[History, Biography, Nonfiction, War, Africa, ..."
6,You Won’t Know I’m Gone,Kristen Orlando,82440.0,55.72,6.90,2.77,1.09,1.68,"[Mystery, Young Adult, Thriller, Fiction, Cont..."
7,Cut,George Pelecanos,68845.0,51.82,7.17,2.13,0.68,1.45,"[Fiction, Crime, Mystery, Thriller, Audiobook,..."
8,Night Prey,John Sandford,96626.0,55.93,7.53,2.21,0.73,1.48,"[Mystery, Fiction, Thriller, Crime, Mystery Th..."
9,Interviewing the Dead,David Field,55861.0,35.57,7.89,3.72,1.61,2.11,"[Historical Fiction, Mystery, Historical Myste..."


In [ ]:
browser.quit()